In [65]:
import os, sys
import re
import jieba, codecs, math

import jieba.posseg as pseg

from pyecharts import options as opts
from pyecharts.charts import Graph

import pyecharts
pyecharts.__version__

'1.7.0'

In [51]:
# 测试按照 句号 切分句子

import re
txt = "【#奔驰女车主不接受4S店道歉# 双方现场交涉言语激烈】4月13日。西安维权奔驰女车主和4S店方见面，双方并未当场和解？4S店相关负责人称：由于出差等原因未及时联系车主!车主则反驳称：可通过电话联系，“没人把你的联系方式给我。”交涉过程中双方言语交锋激烈。#西安奔驰女车主维权# "
#pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
pattern = r'。'
result_list = re.split(pattern, txt)
print(result_list)



['【#奔驰女车主不接受4S店道歉# 双方现场交涉言语激烈】4月13日', '西安维权奔驰女车主和4S店方见面，双方并未当场和解？4S店相关负责人称：由于出差等原因未及时联系车主!车主则反驳称：可通过电话联系，“没人把你的联系方式给我', '”交涉过程中双方言语交锋激烈', '#西安奔驰女车主维权# ']


In [66]:
class RelationExtractor:

    def __init__(self, fpStopWords, fpNameDicts):
        # 人名词典
        self.name_dicts = [line.strip().split(' ')[0] for line in open(fpNameDicts,'rt',encoding='utf-8').readlines()]
        # 停止词表
        self.stop_words = [line.strip() for line in open(fpStopWords,'rt',encoding='utf-8').readlines()]
        #   # 别名词典
        #self.alias_names = dict([(line.split(',')[0].strip(), line.split(',')[1].strip()) for line in open(fpAliasNames,'rt',encoding='utf-8').readlines()])
         # 加载词典
        jieba.load_userdict(fpNameDicts)

    # 提取指定小说文本中的人物关系
    def extract(self, fpText):
        # 人物关系
        relationships = {}
        # 人名频次
        name_frequency = {}
        # 每个段落中的人名
        name_in_paragraph = []

        # 读取小说文本，统计人名出现的频次，以及每个段落中出现的人名
        with codecs.open(fpText, "r", "utf8") as f:
            content = f.read()
            pattern = r'。'
            ## 由于语料文本的组织问题，每行不是一个段落
            ##按照句号分割，10句作为一个段落， 当然这样可能不准确
            result_list = re.split(pattern, content)
            
            for i in range(0,len(result_list),10):
                line = ''.join(result_list[i:i+10])
                # 注意要重新赋值
                line = line.replace('/r/n','')
                
                poss = pseg.cut(line)
                name_in_paragraph.append([])
                
                for w in poss:
                    if w.flag != "nr" or len(w.word) < 2:
                        continue
                    if (w.word in self.stop_words):
                        continue
                    if (not w.word in self.name_dicts ):
                        continue
                    
                    word = w.word
                    # 此处可通过判断增加 姓名规范化 如 小桂子--》韦小宝
                     
                    name_in_paragraph[-1].append(word)
                    if name_frequency.get(word) is None:
                        name_frequency[word] = 0
                        relationships[word] = {}
                    name_frequency[word] += 1

        # 基于共现组织人物关系
        for paragraph in name_in_paragraph:
            for name1 in paragraph:
                for name2 in paragraph:
                    if name1 == name2:
                        continue
                    if relationships[name1].get(name2) is None:
                        relationships[name1][name2] = 1
                    else:
                        relationships[name1][name2] += 1 
        
        # 返回节点和边
        return name_frequency, relationships

    # 输出Gephi格式的节点和边信息
    def exportGephi(self, nodes, relationships):
        # 输出节点
        with codecs.open("./node.txt", "w", "gbk") as f:
            f.write("Id Label Weight\r\n")
            for name, freq in nodes.items():
                f.write(name + " " + name + " " + str(freq) + "\r\n")

        # 输出边
        with codecs.open("./edge.txt", "w", "gbk") as f:
            f.write("Source Target Weight\r\n")
            for name, edges in relationships.items():
                for v, w in edges.items():
                    if w > 0:
                        f.write(name + " " + v + " " + str(w) + "\r\n")   

    # 使用ECharts对人物关系进行渲染
    def exportECharts(self, nodes, relationships):
        # 总频次，用于数据的归一化
        total = sum(list(map(lambda x:x[1], nodes.items())))

        # 输出节点
        nodes_data = []
        for name, freq in nodes.items():
            nodes_data.append(opts.GraphNode(
                name = name, 
                symbol_size = round(freq / total * 100, 2), 
                value = freq,
            )),

        # 输出边
        links_data = []
        for name, edges in relationships.items():
                for v, w in edges.items():
                    if w > 0:
                        links_data.append(opts.GraphLink(
                            source = v, 
                            target = w, 
                            value = w
                        ))

        # 绘制Graph
        g = (
            Graph(init_opts=opts.InitOpts(width = '1440px',height = '800px',theme='westeros'))
            .add(
                "",
                nodes_data,
                links_data,
                gravity = 0.2,
                repulsion = 8000,
                is_draggable = True,
                symbol = 'circle',
                linestyle_opts = opts.LineStyleOpts(
                    curve = 0.3, 
                    width = 0.5, 
                    opacity = 0.7
                ),
                edge_label = opts.LabelOpts(
                    is_show = False, 
                    position = "middle", 
                    formatter = "{b}->{c}"
                ),
            )
            .set_global_opts(
                title_opts = opts.TitleOpts(title="鹿鼎记语义共现关系抽取")
            )
           
        )
        g.render("./鹿鼎记语义共现关系抽取.html")
        print(type(g))
        return g



In [67]:
extractor = RelationExtractor('../../Resources/stopwords/all_stopwords.txt','鹿鼎记人名.txt')
nodes, relationships = extractor.extract('./鹿鼎记.txt')
extractor.exportGephi(nodes, relationships)
g = extractor.exportECharts(nodes, relationships)
g.render_notebook()

<class 'pyecharts.charts.basic_charts.graph.Graph'>


In [54]:
# 测试内容
string_t = ['金庸全集之《鹿鼎记》\r\n作者:金庸\r\nwwww.jb51.net整理制作，并提供下载\r\n\r\n第一回\u3000纵横钩党清流祸\r\n峭茜风期月旦评\r\n北风如刀，满地冰霜', '\r\n江南近海滨的一条大路上，一队清兵手执刀枪，押着七\r\n辆囚车，冲风冒寒，向北而行', '\r\n前面三辆囚车中分别监禁的是三个男子，都作书生打扮，\r\n一个是白发老者，两个是中年人', '后面四辆中坐的是女子，最\r\n后一辆囚车中是个少妇，怀中抱着个女婴', '女婴啼哭不休', '她\r\n母亲温言相呵，女婴只是大哭', '囚车旁一名清兵恼了，伸腿\r\n在车上踢了一脚，喝道：“再哭，再哭！老子踢死你！”那女\r\n婴一惊，哭得更加响了', '\r\n离开道路数十丈处有座大屋，屋檐下站着一个中年文士，\r\n一个十一二岁的小孩', '那文士见到这等情景，不禁长叹一声，\r\n眼眶也红了，说道：“可怜，可怜！”\r\n那小孩子问道：“爹爹，他们犯了什么罪了？”那文士道：\r\n“又犯了什么罪？昨日和今朝，已逮去了三十几人，都是我们\r\n浙江有名的读书人，个个都是无辜株连', '”他说到“无辜株\r\n连”四字，声音压得甚低，生怕给押送囚车的官兵听见了', '那\r\n小孩道：“那个小女孩还在吃奶，难道也犯了罪？真没道理', '”\r\n那文士道：“你懂得官兵没道理，真是好孩子', '唉，人为刀俎，\r\n我为鱼肉，人为鼎镬，我为糜鹿！”\r\n那小孩子道：“爹，你前几天教过我，‘人为刀俎，我为\r\n鱼肉’，就是给人家斩割屠杀的意思', '人家是切菜刀，是砧板，\r\n我们就是鱼和肉', '‘人为鼎镬，我为糜鹿’这两句话，意思也\r\n差不多么？”那文士道：“正是！”眼见官兵和囚车已经去远，\r\n拉着小孩的手道：“外面风大，我们回屋里去', '”当下父子二\r\n人走进书房', '\r\n那文士提笔蘸上了墨，在纸上写了个“鹿”字，说道：\r\n“鹿这种野兽，虽是庞然大物，性子却极为和平，只吃青草树\r\n叶，从来不伤害别的野兽', '凶猛的野兽要伤它吃它，它只有\r\n逃跑，倘若逃不了，那只有给人家吃了', '”又写了“逐鹿”两\r\n字，说道：“因此古人常常拿鹿来比喻天下', '世上百姓都温顺\r\n善良，只有给人欺压残害的份儿', '《汉书》上说：‘秦失其鹿，\r\n天下共逐之', '’那就是说，秦朝失了天下，群雄并起，大家争\r\n夺，最后汉高祖打败了楚霸王，就得了这只又肥又大的鹿', '”\r\n那小孩点头道：“我明白了', '小说书上说‘逐鹿中原’，就\r\n是大家争着要做皇帝的意思', '”那文士甚是喜欢，点了点头，\r\n在纸上画了一只鼎的图形，道：“古人煮食，不用灶头锅子，\r\n用这样三只脚的鼎，下面烧柴，捉到了鹿，就在鼎里煮来吃', '\r\n皇帝和大官都很残忍，心里不喜欢谁，就说他犯了罪，把他\r\n放在鼎里活活煮熟', '《史记》中记载蔺相如对秦王说：‘臣知\r\n欺大王之罪当诛也，臣请就鼎镬', '’就是说：‘我该死，将我\r\n在鼎里烧死了罢！’”\r\n那小孩道：“小说书上又常说‘问鼎中原’，这跟‘逐鹿\r\n中原’好像意思差不多', '”\r\n那文士道：“不错', '夏禹王收九州之金，铸了九口大鼎', '\r\n当时的所谓‘金’其实是铜', '每一口鼎上铸了九州的名字和\r\n山川图形，后世为天下之主的，便保有九鼎', '《左传》上：\r\n‘楚子观兵于周疆', '定王使王孙满劳楚子', '楚子问鼎之大小轻\r\n重焉', '’只有天下之主，方能保有九鼎', '楚王只是楚国的诸侯，\r\n他问鼎的轻重大小，便是心存不轨，想取周王之位而代之', '”\r\n那小孩道：“所以‘问鼎’、‘逐鹿’，便是想做皇帝', '‘未\r\n知鹿死谁手’，就是不知哪一个做成了皇帝', '”\r\n那文士道：“正是', '到得后来，‘问鼎’、‘逐鹿’这四个\r\n字，也可借用于别处，但原来的出典，是专指做皇帝而言', '”\r\n说到这里，叹了口气，道：“咱们做老百姓的，总是死路一条', '\r\n‘未知鹿死谁手’，只不过未知是谁来杀了这头鹿，这头鹿，却\r\n是死定了的', '”\r\n他说着走到窗边，向窗外望去，只见天色阴沉沉地，似\r\n要下雪，叹道：“老天爷何其不仁，数百个无辜之人，在这冰\r\n霜遍地的道上行走', '下起雪来，可又多受一番折磨了', '”\r\n忽见南边大道上两个人戴着斗笠，并肩而来，走到近处，\r\n认出了面貌', '那文士大喜，道：“是你黄伯伯、顾伯伯来啦！”\r\n快步迎将出去，叫道：“梨洲兄、亭林兄，哪一阵好风，吹得\r\n你二位光临？”\r\n右首一人身形微胖，颏下一部黑须、姓黄名宗羲，字梨\r\n洲，浙江余姚人氏', '左首一人又高又瘦，面目黝黑，姓顾名\r\n炎武，字亭林，江苏昆山人氏', '黄顾二人都是当世大儒，明\r\n亡之后，心伤国变，隐居不仕，这日连袂来到崇德', '顾炎武\r\n走上几步，说道：“晚村兄，有一件要紧事，特来和你商议', '”\r\n这文士姓吕名留良，号晚村，世居浙江杭州府崇德县，也\r\n是明末、清初一位极有名的隐士', '他眼见黄顾二人脸色凝重，\r\n又知顾炎武向来极富机变，临事镇定，既说是要紧事，自然\r\n非同小可，拱手道：“两位请进去先喝三杯，解解寒气', '”当\r\n下请二人进屋，吩咐那小孩道：“葆中，去跟娘说，黄伯伯、\r\n顾伯伯到了，先切两盘羊膏来下酒', '”\r\n不多时，那小孩吕葆中和兄弟毅中搬出三副杯筷，布在\r\n书房桌上', '一名老仆奉上酒菜', '吕留良待三人退出，关上了\r\n书房门，说道：“黄兄，顾兄，先喝三杯！”\r\n黄宗羲神色惨然，摇了摇头', '顾炎武却自斟自饮，一口\r\n气连干了六杯', '\r\n吕留良道：“二位此来，可是和‘明史’一案有关吗？”黄\r\n宗羲道：“正是！”顾炎武提起酒杯，高声吟道：“清风虽细难\r\n吹我，明月何尝不照人？’晚村兄，你这两句诗，真是绝唱！\r\n我每逢饮酒，必诵此诗，必浮大白', '”\r\n吕留良心怀故国，不肯在清朝做官', '当地大吏仰慕他声\r\n名，保荐他为“山林隐逸”，应征赴朝为官，吕留良誓死相拒，\r\n大吏不敢再逼', '后来又有一名大官保荐他为“博学鸿儒”，吕\r\n留良眼见若再相拒，显是轻侮朝廷，不免有杀身之祸，于是\r\n削发为僧，做了假和尚', '地方官员见他意坚，从此不再劝他\r\n出山', '“清风、明月”这两句诗，讥刺满清，怀念前明，虽然\r\n不敢刊行，但在志同道合的朋辈之间传诵已遍，此刻顾炎武\r\n又读了出来', '黄宗羲道：“真是好诗！”举起酒杯，也喝了一\r\n杯', '吕留良道：“两位谬赞了', '”\r\n顾炎武一抬头，见到壁上挂着一幅高约五尺、宽约丈许\r\n的大画，绘的是一大片山水，笔势纵横，气象雄伟，不禁喝\r\n了声彩，画上只题了四个大字：“如此江山”，说道：“看这笔\r\n路，当是二瞻先生的丹青了', '”吕留良道：“正是', '”那“二\r\n瞻”姓查，名士标，是明末清初的一位大画家，也和顾黄吕\r\n诸人交好', '黄宗羲道：“这等好画，如何却无题跋？”吕留良\r\n叹道：“二瞻先生此画，颇有深意', '只是他为人稳重谨慎，既\r\n不落款，亦无题跋', '他上个月在舍间盘桓，一时兴到，画了\r\n送我，两位便题上几句如何？”\r\n顾黄二人站起身来，走到画前仔细观看，只见大江浩浩\r\n东流，两岸峰峦无数，点缀着奇树怪石，只是画中云气瀰漫，\r\n山川虽美，却令人一见之下，胸臆间顿生郁积之意', '\r\n顾炎武道：“如此江山，沦于夷狄', '我辈忍气吞声，偷生\r\n其间，实令人悲愤填膺', '晚村兄何不便题诗一首，将二瞻先\r\n生之意，表而出之？”吕留良道：“好！”当即取下画来，平铺\r\n于桌', '黄宗羲研起了墨', '吕留良提笔沉吟半晌，便在画上振\r\n笔直书', '顷刻诗成，诗云：\r\n“其为宋之南渡耶？如此江山真可耻', '其为崖山以后耶？\r\n如此江山不忍视', '吾今始悟作画意，痛哭流涕有若是', '以今\r\n视昔昔犹今，吞声不用枚衔嘴', '画将桌羽西台泪，研入丹青\r\n提笔泚', '所以有画无诗文，诗文尽在四字里', '尝谓生逢洪武\r\n初，如瞽忽瞳跛可履', '山川开霁故璧完，何处登临不狂喜？”\r\n书完，掷笔于地，不禁泪下', '\r\n顾炎武道：“痛快淋漓，真是绝妙好辞', '”吕留良道：“这\r\n诗殊无含蓄，算不得好，也只是将二瞻先生之原意写了出来，\r\n好教观画之人得知', '”黄宗羲道：“何日故国重光，那时‘山\r\n川开霁故璧完’，纵然是穷山恶水，也令人观之大畅胸怀，真\r\n所谓‘何处登临不狂喜’了！”顾炎武道：“此诗结得甚妙！终\r\n有一日驱除胡虏，还我大汉山河，比之徒抒悲愤，更加令人\r\n气壮', '”\r\n黄宗羲慢慢将画卷了起来，说道：“这画是挂不得了，晚\r\n村兄须得妥为收藏才是', '倘若给吴之荣之类奸人见到，官府\r\n查究起来，晚村兄固然麻烦，还牵累了二瞻先生', '”\r\n顾炎武拍桌骂道：“吴之荣这狗贼，我真恨不得生食其\r\n肉', '”吕留良道：“二位枉顾，说道有件要紧事', '我辈书生积\r\n习，作诗题画，却搁下了正事', '不知究是如何？”黄宗羲道：\r\n“我二人此来，乃是为了二瞻先生那位本家伊璜先生', '小弟和\r\n顾兄前日得到讯息，原来这场‘明史’大案，竟将伊璜先生\r\n也牵连在内', '”吕留良惊道：“伊璜兄也受了牵连？”\r\n黄宗羲道：“是啊', '我二人前日晚上匆匆赶到海宁袁花镇，\r\n伊璜先生并不在家，说是出外访友去了', '炎武兄眼见事势紧\r\n急，忙嘱伊璜先生家人连夜躲避；想起伊璜先生和晚村兄交\r\n好，特来探访', '”吕留良道：“他……他却没有来', '不知到了\r\n何处', '”顾炎武道：“他如在府上，这会儿自已出来相见', '我\r\n已在他书房的墙壁上题诗一首，他若归家，自然明白，知所\r\n趋避，怕的是不知讯息，在外露面，给公人拿住，那可糟了', '”\r\n黄宗羲道：“这‘明史’一案，令我浙西名士几乎尽遭毒\r\n手', '清廷之意甚恶，晚村兄名头太大，亭林兄与小弟之意，要\r\n劝晚村兄暂且离家远游，避一避风头', '”\r\n吕留良气愤愤的道：“鞑子皇帝倘若将我捉到北京，拚着\r\n千刀万剐，好歹也要痛骂他一场，出了胸中这口恶气，才痛\r\n痛快快的就死', '”\r\n顾炎武道：“晚村兄豪气干云，令人好生钦佩', '怕的是见\r\n不到鞑子皇帝，却死于一般下贱的奴才手里', '再说，鞑子皇\r\n帝只是个小孩子，什么也不懂，朝政大权，尽操于权臣鳌拜\r\n之手', '兄弟和梨洲兄推想，这次‘明史’一案所以如此大张\r\n旗鼓，雷厉风行，当是鳌拜意欲挫折我江南士人之气', '”\r\n吕留良道：“两位所见甚是', '清兵入关以来，在江北横行\r\n无阻，一到江南，却处处遇到反抗，尤其读书人知道华夷之\r\n防，不断跟他们捣蛋', '鳌拜乘此机会，要对我江南士子大加\r\n镇压', '哼，野火烧不尽，春风吹又生，除非他把咱们江南读\r\n书人杀得干干净净', '”\r\n黄宗羲道：“是啊', '因此咱们要留得有用之身，和鞑子周\r\n旋到底，倘若逞了一时血气之勇，反是堕入鞑子的算中了', '”\r\n吕留良登时省悟，黄顾二人冒寒枉顾，一来固是寻觅查\r\n伊璜，二来是劝自己出避，生怕自己一时按捺不住，枉自送\r\n了性命，良友苦心，实深感激，说道：“二位金石良言，兄弟\r\n哪敢不遵？明日一早，兄弟全家便出去避一避', '”黄顾二人大\r\n喜，齐声道：“自该如此', '”\r\n吕留良沉吟道：“却不知避向何处才好？”只觉天涯茫茫，\r\n到处是鞑子的天下，真无一片干净土地，沉吟道：“桃源何处，\r\n可避暴秦？桃源何处，可避暴秦？”顾炎武道：“当今之世，便\r\n真有桃源乐土，咱们也不能独善其身，去躲了起来……”吕\r\n留良不等他辞毕，拍案而起，大声道：“亭林兄此言责备得是', '\r\n国家兴亡，匹夫有责，暂时避祸则可，但若去躲在桃花源里，\r\n逍遥自在，忍令亿万百姓在鞑子铁蹄下受苦，于心何安？兄\r\n弟失言了', '”\r\n顾炎武微笑道：“兄弟近年浪迹江湖，着实结交了不少朋\r\n友', '大江南北，见闻所及，不但读书人反对鞑子，而贩夫走\r\n卒、屠沽市井之中，也到处有热血满腔的豪杰', '晚村兄要是\r\n有意，咱三人结伴同去扬州，兄弟给你引见几位同道中人如\r\n何？”吕留良大喜，道：“妙极，妙极！咱们明日便去扬州，二\r\n位少坐，兄弟去告知拙荆，让她收拾收拾', '”说着匆匆入内', '\r\n不多时吕留良回到书房，说道：“‘明史’一案，外间虽\r\n传说纷纷，但一来传闻未必确实，二来说话之人又顾忌甚多，\r\n不敢尽言', '兄弟独处蜗居，未知其详，到底是何起因？”\r\n顾炎武叹了口气，道：“这部明史，咱们大家都是看过的\r\n了，其中对鞑子不大恭敬，那也是有的', '此书本是出于我大\r\n明朱国桢相国之手，说到关外建州卫之事，又如何会对鞑子\r\n客气？”吕留良点头道：“听说湖州庄家花了几千两银子，从\r\n朱相国后人手中将明史原稿买了来，以己名刊行，不想竟然\r\n酿此大祸', '”\r\n浙西杭州、嘉兴、湖州三府，处于太湖之滨，地势平坦，\r\n土质肥沃，盛产稻米蚕丝', '湖州府的首县今日称为吴兴县，清\r\n时分为乌程、归安两县', '自来文风甚盛，历代才士辈出，梁\r\n时将中国字分为平上去入四声的沈约，元代书画皆臻极品的\r\n赵孟*，都是湖州人氏', '当地又以产笔著名，湖州之笔，徽\r\n州之墨，宣城之纸，肇庆端溪之砚，文房四宝，天下驰名', '\r\n湖州府有一南浔镇，虽是一个镇，却比寻常州县还大，镇\r\n上富户极多，著名的富室大族之中有一家姓庄', '其时庄家的\r\n富户名叫庄允城，生有数子，长子名叫廷鑨，自幼爱好诗书，\r\n和江南名士才子多所结交', '到得顺治年间，庄廷鑨因读书过\r\n勤，忽然眼盲，寻遍名医，无法治愈，自是郁郁不欢', '\r\n忽有一日，邻里有一姓朱的少年携来一部手稿，说是祖\r\n父朱相国的遗稿，向庄家抵押，求借数百两银子', '庄家素来\r\n慷慨，对朱相国的后人一直照顾，既来求借，当即允诺，也\r\n不要他用什么遗稿抵押', '但那姓朱少年说道借得银子之后，要\r\n出门远游，这部祖先的遗稿带在身边，恐有遗失，存在家里\r\n又不放心，要寄存在庄家', '庄允城便答应了', '那姓朱少年去\r\n后，庄允城为替儿子解闷，叫家中清客读给他听', '\r\n朱国桢这部明史稿，大部分已经刊行，流传于世，这次\r\n他孙子携来向庄家抵押的，是最后的许多篇列传', '庄廷鑨听\r\n清客读了数日，很感兴味，忽然想起：“昔时左丘明也是盲眼\r\n之人，却因一部史书《左传》，得享大名于千载之后', '我今日\r\n眼盲，闲居无聊，何不也撰述一部史书出来，流传后世？”\r\n大富之家，办事容易，他既兴了此念，当即聘请了好几\r\n位士人，将那部明史稿从头至尾的读给他听', '他认为何处当\r\n增，何处当删，便口述出来，由宾客笔录', '\r\n但想自己眼盲，无法博览群籍，这部明史修撰出来，如\r\n内容谬误甚多，不但大名难享，反而被人讥笑，于是又花了\r\n大批银两，延请许多通士鸿儒，再加修订，务求尽善尽美', '有\r\n些大有学问之人非钱财所能请到，庄廷鑨便辗转托人，卑辞\r\n相邀', '太湖之滨向来文士甚多，受到庄家邀请的，一来怜其\r\n眼盲，感其意诚；二来又觉修撰明史乃是一件美事，大都到\r\n庄家来作客十天半月，对稿本或正其误，或加润饰，或撰写\r\n一两篇文字', '因此这部明史确是集不少大手笔之力', '书成不\r\n久，庄廷鑨便即去世', '\r\n庄允城心伤爱子之逝，即行刊书', '清代刊印一部书，着\r\n实不易，要招请工匠，雕成一块块木版，这才印刷成书', '这\r\n部明史卷帙浩繁，雕工印工，费用甚巨', '好在庄家有的是钱，\r\n拨出几间大屋作为工场，多请工匠，数年间便将书刊成了，书\r\n名叫作《明书辑略》，撰书人列名为庄廷鑨，请名士李令晰作\r\n序', '所有曾经襄助其事的学者也都列名其上，有茅元锡、吴\r\n之铭、吴之熔、李祈涛、茅次莱、吴楚、唐元楼、严云起、蒋\r\n麟徵、韦金祐、韦一园、张隽、董二酉、吴炎、潘柽章、陆\r\n圻、查继佐、范骧等，共一十八人', '书中又提到此书是根据\r\n朱氏的原稿增删而成，不过朱国桢是明朝相国，名头太大，不\r\n便直书其名，因此含含糊糊的只说是“朱氏原稿”', '\r\n《明书辑略》经过这许多文人学士撰改修订，是以体例精\r\n备，叙述详明，文字又华瞻雅致，书出后大获士林赞誉', '庄\r\n家又是志在扬名，书价取得极廉', '原稿中涉及满洲之时，本\r\n有不少攻讦指摘的言语，修史诸人早已一一删去，但赞扬明\r\n朝的文字却也在所不免', '当时明亡未久，读书人心怀故国，书\r\n一刊行，立刻就大大畅销', '庄廷鑨之名噪于江北江南', '庄允\r\n城虽有丧子之痛，但见儿子成名于身后，自是老怀弥慰', '\r\n也是乱世之时，该当小人得志，君子遭祸', '湖州归安县\r\n的知县姓吴名之荣，在任内贪赃枉法，百姓恨之切齿，终于\r\n为人告发，朝廷下令革职', '吴之荣做了一任归安县知县，虽\r\n然搜刮了上万两银子，但革职的廷令一下，他东贿西赂，到\r\n处打点，才免得抄家查办的处分，这上万两赃款却也已荡然\r\n无存，连随身家人也走得不知去向', '他官财两失，只得向各\r\n家富室一处处去打秋风，说道为官清苦，此番丢官，连回家\r\n也没有盘缠，无法成行', '有些富人为免麻烦，便送他十两八\r\n两银子', '待得来到富室朱家，主人朱佑明却是个嫉恶如仇的\r\n正直君子，非但不送仪程，反而狠狠讥刺，说道阁下在湖州\r\n做官，百姓给你害得好苦，我朱某就算有钱，也宁可去周济\r\n给阁下害苦了的贫民', '吴之荣虽然恼怒，却也无法可施，他\r\n既已被革职，无权无势，又怎能再奈何得了富家巨室？当下\r\n又来拜访庄允城', '\r\n庄允城平素结交清流名士，对这赃官很瞧不起，见他到\r\n来求索，冷笑一声，封了一两银子给他，说道：“依阁下的为\r\n人，这两银子本是不该送的，只是湖州百姓盼望阁下早去一\r\n刻好一刻，多一两银子，能早去片刻，也是好的', '”\r\n吴之荣心下怒极，一瞥眼见到大厅桌上放得有一部《明\r\n书辑略》，心想：“这姓庄的爱听奉承，人家只要一赞这部明\r\n史修得如何如何好，白花花的银子双手捧给人家，再也不皱\r\n一皱眉头', '”便笑道：“庄翁厚赐，却之不恭', '兄弟今日离别\r\n湖州，最遗憾的便是无法将‘湖州之宝’带一部回家，好让\r\n敝乡孤陋寡闻之辈大开眼界', '”\r\n庄允城问道：“什么叫做‘湖州之宝’？”吴之荣笑道：\r\n“庄翁这可太谦了', '士林之中，纷纷都说，令郎廷鑨龙公子亲\r\n笔所撰的那部《明书辑略》，史才、史识、史笔，无一不是旷\r\n古罕有，左马班庄，乃是古今良史四大家', '这‘湖州之宝’，\r\n自然便是令郎亲笔所撰的明史了', '”\r\n吴之荣前一句“令郎亲笔所撰”，后一句“令郎亲笔所\r\n撰”，把庄允城听得心花怒放', '他明知此书并非儿子亲作，内\r\n心不免遗憾，吴之荣如此说，正是大投所好，心想：“人家都\r\n说此人贪赃，是个龌龊小人，但他毕竟是个读书人，眼光倒\r\n是有的', '原来外间说鑨儿此书是‘湖州之宝’，这话倒是第一\r\n次听见', '”不由得笑容满脸，说道：“荣翁说什么左马班庄，古\r\n今四大良史，兄弟可不大明白，还请指教', '”吴之荣见他脸色\r\n顿和，知道马屁已经拍上，心下暗暗喜欢', '说道：“庄翁未免\r\n太谦了', '左丘明作《左传》，司马迁作《史记》，班固作《汉\r\n书》，都是传诵千载的名作，自班固而后，大史家就没有了', '\r\n欧阳修作《五代史》，司马光作《资治通鉴》，文章虽佳，才\r\n识终究差了', '直到我大清盛世，令郎亲笔所撰这部煌煌巨作\r\n《明书辑略》出来，方始有人能和左丘明、司马迁、班固三位\r\n前辈并驾齐驱，‘四大良史，左马班庄’，这句话便是由此而\r\n生', '”\r\n庄允城笑容满面，连连拱手，说道：“谬赞，谬赞！不过\r\n“湖州之宝’这句话，毕竟当不起', '”吴之荣正色道：“怎么当\r\n不起？外间大家都说：‘湖州之宝史丝笔，还是庄史居第一’！”\r\n蚕丝和毛笔是湖州两大名产，吴之荣品格卑下，却有三分才\r\n情，出口成章，将“庄史”和湖丝、湖笔并称', '庄允城听得\r\n更是喜欢', '\r\n吴之荣又道：“兄弟来到贵处做官，两袖清风，一无所得', '\r\n今日老着脸皮，要向庄翁求一部明史，作为我家传家之宝', '日\r\n后我吴家子孙日夕诵读，自必才思大进，光宗耀祖，全仗庄\r\n翁之厚赐了', '”庄允城笑道：“自当奉赠', '”吴之荣又谈了几句，\r\n不见庄允城有何举动，当下又将这部明史大大恭维了一阵，其\r\n实这部书他一页也未读过，只是史才如何如何了得，史识又\r\n如何如何超卓，不着边际的瞎说', '庄允城道：“荣翁且请宽坐', '”\r\n回进内堂', '\r\n过了良久，一名家丁捧了一个包裹出来，放在桌上', '吴\r\n之荣见庄允城尚未出来，忙将包裹掂了一掂，那包裹虽大，却\r\n是轻飘飘地，内中显然并无银两，心下好生失望', '过得片刻，\r\n庄允城回到厅上，捧起包裹，笑道：“荣翁瞧得起敝处的土产，\r\n谨以相赠', '”\r\n吴之荣谢了，告辞出来，没回到客店，便伸手到包裹中\r\n一阵掏摸，摸到的竟是一部书，一束蚕丝，几十管毛笔', '他\r\n费了许多唇舌，本想庄允城在一部明史之外，另有几百两银\r\n子相赠，可是赠送的竟是他信口胡诌的“湖州三宝”，心下暗\r\n骂：“他妈的，南浔这些财主，都如此小气！也是我说错了话，\r\n倘若我说湖州三宝乃是金子银子和明史，岂不是大有所获？”\r\n气愤愤的回到客店，将包裹往桌上一丢，倒头便睡，一\r\n觉醒来，天已大黑，客店中吃饭的时候已过，他又舍不得另\r\n叫饭菜，愁肠饥火，两相煎熬，再也睡不着觉，当下解开包\r\n裹，翻开那部《明书辑略》阅看', '看得几页，眼前金光一闪，\r\n赫然出现一张金叶', '吴之荣一颗心怦怦乱跳，揉了揉眼细看，\r\n却不是金叶是什么？当下一阵乱抖，从书中抖了十张金叶出\r\n来，每一张少说也有五钱，十张金叶便有五两黄金', '其时金\r\n贵，五两黄金抵得四百两银子', '\r\n吴之荣喜不自胜，寻思：“这姓庄的果然狡狯，他怕我讨\r\n得这部书去，随手抛弃，翻也不翻，因此将金叶子夹在书中，\r\n看是谁读他儿子这部书，谁便有福气得此金叶']

In [55]:
for x in range(0,len(string_t),10):
    print(x)
    line = ''.join(string_t[x:x+10])
    line = line.replace('\r\n','')
    print(line)

0
金庸全集之《鹿鼎记》作者:金庸wwww.jb51.net整理制作，并提供下载第一回　纵横钩党清流祸峭茜风期月旦评北风如刀，满地冰霜江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人后面四辆中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴女婴啼哭不休她母亲温言相呵，女婴只是大哭囚车旁一名清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！老子踢死你！”那女婴一惊，哭得更加响了离开道路数十丈处有座大屋，屋檐下站着一个中年文士，一个十一二岁的小孩那文士见到这等情景，不禁长叹一声，眼眶也红了，说道：“可怜，可怜！”那小孩子问道：“爹爹，他们犯了什么罪了？”那文士道：“又犯了什么罪？昨日和今朝，已逮去了三十几人，都是我们浙江有名的读书人，个个都是无辜株连”他说到“无辜株连”四字，声音压得甚低，生怕给押送囚车的官兵听见了
10
那小孩道：“那个小女孩还在吃奶，难道也犯了罪？真没道理”那文士道：“你懂得官兵没道理，真是好孩子唉，人为刀俎，我为鱼肉，人为鼎镬，我为糜鹿！”那小孩子道：“爹，你前几天教过我，‘人为刀俎，我为鱼肉’，就是给人家斩割屠杀的意思人家是切菜刀，是砧板，我们就是鱼和肉‘人为鼎镬，我为糜鹿’这两句话，意思也差不多么？”那文士道：“正是！”眼见官兵和囚车已经去远，拉着小孩的手道：“外面风大，我们回屋里去”当下父子二人走进书房那文士提笔蘸上了墨，在纸上写了个“鹿”字，说道：“鹿这种野兽，虽是庞然大物，性子却极为和平，只吃青草树叶，从来不伤害别的野兽凶猛的野兽要伤它吃它，它只有逃跑，倘若逃不了，那只有给人家吃了”又写了“逐鹿”两字，说道：“因此古人常常拿鹿来比喻天下世上百姓都温顺善良，只有给人欺压残害的份儿
20
《汉书》上说：‘秦失其鹿，天下共逐之’那就是说，秦朝失了天下，群雄并起，大家争夺，最后汉高祖打败了楚霸王，就得了这只又肥又大的鹿”那小孩点头道：“我明白了小说书上说‘逐鹿中原’，就是大家争着要做皇帝的意思”那文士甚是喜欢，点了点头，在纸上画了一只鼎的图形，道：“古人煮食，不用灶头锅子，用这样三只脚的鼎，下面烧柴，捉到了鹿，就在鼎里煮来吃皇帝和大官都很残忍，心里不喜欢谁，就说他犯了罪，把他放在鼎里活活煮熟《史记》中记载蔺相如对秦王说：‘臣知欺大王之罪当诛也